In [1]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/{search} 뉴스 제목{num}.csv',encoding="utf-8")

# df = pd.read_csv(f'csv/kakao_data_label.csv',encoding='cp949') # encoding='cp949'
# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# df['content'] = df['title']

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 777


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [2]:
# csv 파일로 저장
df.to_csv(f'csv/sentiment_{search}_{num}.csv', index=False)

# 결과 확인
print(df['title'])
print(df['sentiment'])

0                  [삼성전자, ,, 올림픽, 의, 날, 앞두, 고, 운동, 캠페인]
1     [삼성전자, ,, ', AI, 로, 에너지, 절약, 하, 는, ', 비스포크, 신제...
2        [삼성전자, ,, 전, 세계, 에, '비스포크, 라이프, ', 가전, 늘, 리ㄴ다]
3     [[, 특징주, ], 삼성전자, ,, 3거래, 이, ㄹ, 연속, 하락…아슬한, ‘7...
4     [[, 단독, ], 한진만, 삼성전자, 미주총괄, 부사장, ", 美, 칩, 인력, ...
5     [삼성전자, ,, 3거래, 이, ㄹ, 연속, 하락세…‘7만전자’, 턱걸이, [, 특...
6             [반도체, 패권, 잡자…, 삼성전자·TSMC, ', 기술경쟁, ', 격화]
7     [현대차, 에, 삼성전자, ', 최첨단, 두뇌, ', 심, 는다, [, 삼성, -,...
8     [삼성전자, 시스템, 반도체, 가, 들, 어, 가, ㄴ, 현대차, 를, 트, 어, ...
9       [이재, 용, ·, 정의선, 손잡았다…삼성전자, 車반도체, ,, 현대차, 에, 탑재]
10                [경계현, 삼성전자, 사장, 과, 인사, 하, 는, 김기현, 대표]
11               [참석자, 와, 대화, 하, 는, 경계현, 삼성전자, DS, 부문장]
12                 [', 삼성전자, 와, 이탈리아, 토일렛페이퍼, 의, 만남, ']
13    [[스페셜리포트]삼성전자·LGD, ‘OLED, 동맹’, ..., 韓, 대형, 디스플...
14    [삼성전자, ,, ', 글로벌, 비스포크, 라이프, 2023, ', 온라인, 행사,...
15    [삼성전자, 만, 10조, 이상, 폭풍매수…외국, 이, ㄴ, 쏠림현상, 에, 증시,...
16        [삼성전자, ,, 현대차, 에, 차량용칩, 공급…인포테인먼트, 분야, 첫, 협력]
17    [美中, 반도체戰, 에, 끼, ㄴ, 삼성전자…‘7만전자’,, 엔비디아, ‘조

In [ ]:
df